In [1]:
import hs_utils
from hs_restclient import HydroShare, HydroShareAuthBasic, HydroShareHTTPException

In [2]:
h = hs_utils.hydroshare()

In [5]:
h.getSecureConnection(email = 'castronova.anthony@gmail.com')

Enter you HydroShare Password: ········


In [6]:
h.getResourceContent('1a43f21caa9941d6b79dd53565301175')

HydroShareArgumentException: http://172.17.0.2/user/jovyan/tree/notebooks/data/. is not a directory.